In [60]:
import boto3
import os

# Get the service resource 使用boto3連線sqs

sqs = boto3.resource('sqs',region_name=os.environ["region_name"],
                      aws_access_key_id=os.environ["aws_access_key_id"],
                      aws_secret_access_key=os.environ["aws_secret_access_key"],
                      endpoint_url="http://cc104.sqs.local:9324")

# Create the queue. This returns an SQS.Queue instance 使用sqs.create_queue創一個叫test的隊列 delaysecond為5秒
queue = sqs.create_queue(QueueName='test', Attributes={'DelaySeconds': '5'})

# You can now access identifiers and attributes
print("Url:",queue.url)
print("DelaySeconds:",queue.attributes.get('DelaySeconds'))

Url: http://cc104.sqs.local:9324/queue/test
DelaySeconds: 5


In [119]:
# Get the queue 使用sqs.get_queue_by_name讀取test隊列內容
queue = sqs.get_queue_by_name(QueueName='test')

# Create a new message #用send_message方法將message傳送到test隊列中(法一：一次送一個)
response = queue.send_message(MessageBody='LBJ')

# The response is NOT a resource, but gives you a message ID and MD5 
print("MessageId:",response.get('MessageId'))
print("MD5OfMessageBody:",response.get('MD5OfMessageBody'))

MessageId: 155fecf1-f5db-492e-a276-39d765d5679b
MD5OfMessageBody: a3aac5097340a9aa8c27278a39971223


In [122]:
#用send_message方法將message傳送到test隊列中(法一：一次送多個)
response = queue.send_messages(Entries=[
    {
        'Id': '1',
        'MessageBody': 'GGboy'
    },
    {
        'Id': '2',
        'MessageBody': 'boto3',
        'MessageAttributes': {
            'Author': {
                'StringValue': 'Daniel',
                'DataType': 'String'
            }
        }
    }
])
# Print out any failures 假如有失敗的印出來
print(response.get('Failed'))

None


In [124]:
#查詢messages(一次只能查到一個)
for message in queue.receive_messages(MessageAttributeNames=['Author']):
    # Get the custom author message attribute if it was set
    author_text = ''
    if message.message_attributes is not None:
        author_name = message.message_attributes.get('Author').get('StringValue')
        if author_name:
            author_text = ' ({0})'.format(author_name)

    # Print out the body and author (if set)
    print('{0}{1},Winner, Winner, Chicken Dinner'.format(message.body, author_text))
    message.delete()

GGboy,Winner, Winner, Chicken Dinner
